In [1]:
import pandas as pd
import numpy as np
import datetime
import gc
import math

import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
#import matplotlib.pyplot as plt
#import seaborn as sns

In [2]:
%%time

usecols=['Semana', 'Cliente_ID', 'Producto_ID', 'Demanda_uni_equil']

dtype = {'Semana':'category',
         'Canal_ID':'uint8',
         'Agencia_ID':'uint16',
         'Ruta_SAK':'uint16',
         'Dev_uni_proxima':'uint16',
         'Cliente_ID':'category',
         'Producto_ID':'category',
         'Venta_uni_hoy':'uint32',
         'Demanda_uni_equil':'uint16',
         'Venta_hoy':'float32',
         'Dev_proxima':'float32'}
try:
    df
except NameError:
    print('Carregando 1ª vez!!')
else:
    print('Limpando e carregando de novo!')
    del df
    gc.collect()
    
finally:
    df = pd.concat([pd.read_csv('train_3.csv', dtype=dtype, usecols=usecols),
                    pd.read_csv('train_5.csv', dtype=dtype, usecols=usecols),
                    pd.read_csv('train_6.csv', dtype=dtype, usecols=usecols),
                    pd.read_csv('train_8.csv', dtype=dtype, usecols=usecols)
                   ])
   
    



Carregando 1ª vez!!
Wall time: 40.5 s


In [31]:
df[['Cliente_ID','Producto_ID']].set_index(['Cliente_ID','Producto_ID']).head()

Empty DataFrame
Columns: []
Index: [(15766, 1212), (15766, 1216), (15766, 1238), (15766, 1240), (15766, 1242)]

In [41]:
df.groupby(['Producto_ID']).agg(['mean', 'median', 'count','std'])

Demanda_uni_equil                               
                         mean  median     count          std
Producto_ID                                                 
1                 1858.748784  1261.0  19689836  1123.200164
100                  7.727273     3.5        22    10.682348
1039                 9.588315     4.0       736    16.854328
106                  5.894958     3.0       476     8.511138
1064                 2.216536     2.0    101706     1.989121
107                  2.800000     2.0         5     3.114482
108                 73.684211    24.0        19   102.504771
11                4401.320070  3917.0    272078   898.433090
1109                 3.302861     2.0    191804     5.205433
1111                99.500000    94.0        10    83.281651
1112                37.214407    24.0      1180    45.392556
1120                17.758084     8.0     12587    34.824746
1125                 8.382909     5.0    200019    15.296579
1126                 8.778963     4.0      1968    14.501246
1129                 5.630919     3.0     81142    10.813552
1134                 5.785795     4.0      6167     8.663643
1143                68.285714    44.0         7    60.722632
1144                10.259933     6.0      9287    16.066154
1145                14.028837     6.0     12276    29.617028
1146                 4.422088     2.0    210878    15.729220
1147                 2.906817     2.0      4078     3.377797
1148                 4.676895     3.0       554     5.873157
1150                 6.389279     2.0    148143    16.743603
1153                35.789474    39.0        19    22.230057
1156               120.875000    86.5         8   108.678343
1160                 4.616478     2.0     79002    12.594105
1164                35.400000    38.0         5     6.655825
1166                45.297710     9.0       131   401.752865
1167                53.233083    18.0       133   398.092305
1168                42.442748    12.0       131   334.433164
...                       ...     ...       ...          ...
73                   3.221337     2.0     31495     7.761949
7340                 6.797196     5.0      1070     8.777842
7342                50.000000    15.0         7    67.626425
739                  4.455696     2.0       395    12.163853
7724                86.929712    84.0       313    55.264007
7761               301.741935    50.0        31   613.884787
779                  4.949102     2.0       334     8.445219
7814               147.752412   141.0       311    85.237125
7829                81.760766    75.0       209    71.331365
8                 4126.002737  3419.0     19364  1040.039052
815                  4.290363     3.0      3227     5.995269
8768                 1.250000     1.0         8     0.462910
8825                86.538462    50.0        13   120.557466
8863                11.298013    10.0       302     7.722164
8864                47.986333    44.0       439    40.729644
8865                15.902141    14.0       327    11.322499
890                 33.800000    23.5        10    25.650644
8921                 6.403118     5.0     14497     5.763793
8931                13.209835    10.0      7463    11.808268
8940                 7.213395     5.0     13796     7.928723
9                 3411.853211  3406.0       109    46.584019
9082                 4.393939     3.0        99     4.027503
9217                 3.856433     3.0     44063     4.213310
9248                61.122066    60.0       213    40.286054
9416               454.000000    95.0         5   630.787207
9711                10.500000     6.5        22     7.750576
9717                11.136364     6.0        88    15.468308
972                  2.478903     2.0    101342     2.441554
9753                40.533333    10.0        15   123.312534
9756                11.333333     6.0         3    11.015141

[1578 rows x 4 columns]

Considerar apenas o produto parece não ser a melhor abordagem. O desvio padrão em alguns itens está muito alto. Talvez ajude se for feita a mesma agregação por cliente e produto:

Fazendo a média de 3 semanas para cliente e produto

In [3]:
gb = df.groupby(['Cliente_ID','Producto_ID']).mean().reset_index()

In [4]:
gb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11141466 entries, 0 to 11141465
Data columns (total 3 columns):
Cliente_ID           object
Producto_ID          object
Demanda_uni_equil    float64
dtypes: float64(1), object(2)
memory usage: 255.0+ MB


In [63]:
gb.head()

,Cliente_ID,Producto_ID,Demanda_uni_equil
0,100000,43274,1.0
1,1000001,1220,2.0
2,1000001,1240,11.0
3,1000001,1242,2.0
4,1000001,1250,5.0


Montando XGB regression

In [47]:
X = gb.iloc[:,0:2]
y = gb.iloc[:,-1]

- using rmsle: https://www.kaggle.com/kobakhit/xgboost (in comments)

In [48]:
# transformations
X = MinMaxScaler().fit_transform(X)

#y = y.apply(lambda item: math.log(1+item))

In [27]:
del df
gc.collect()

NameError: name 'df' is not defined

In [49]:
x1, x2, y1, y2 = train_test_split(X, y, test_size=0.1, random_state=99)

In [50]:
del X, y
gc.collect()

473

<br>[10:28:52] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'
<br>[0]	train-rmse:1.33005	valid-rmse:1.3307
<br>[15]	train-rmse:1.06597	valid-rmse:1.06658
<br>[30]	train-rmse:0.914119	valid-rmse:0.914639
<br>[45]	train-rmse:0.829388	valid-rmse:0.829772
<br>[60]	train-rmse:0.786017	valid-rmse:0.78631
<br>[75]	train-rmse:0.761699	valid-rmse:0.761935
<br>[90]	train-rmse:0.749326	valid-rmse:0.749516
<br>[105]	train-rmse:0.739717	valid-rmse:0.739841
<br>[120]	train-rmse:0.735601	valid-rmse:0.735698
<br>[135]	train-rmse:0.732291	valid-rmse:0.732354
<br>[149]	train-rmse:0.729319	valid-rmse:0.729351

In [51]:
params = {
    'objective' : "reg:linear",
    'eta' : 0.025,
    'max_depth' : 5,
    'subsample' : 0.8,
    'colsample_bytree' : 0.6,
    'silent' : True,
    'eval_metric': 'rmse', 
}

watchlist= [(xgb.DMatrix(x1, y1), 'train'), (xgb.DMatrix(x2, y2), 'valid')]
model = xgb.train(params, xgb.DMatrix(x1, y1), 200, watchlist, maximize=True, verbose_eval=20)

[09:28:16] Tree method is automatically selected to be 'approx' for faster speed. to use old behavior(exact greedy algorithm on single machine), set tree_method to 'exact'
[0]	train-rmse:36.3761	valid-rmse:36.6562
[20]	train-rmse:28.6083	valid-rmse:29.0346
[40]	train-rmse:25.076	valid-rmse:25.519
[60]	train-rmse:23.1628	valid-rmse:23.5417
[80]	train-rmse:22.0437	valid-rmse:22.3429
[100]	train-rmse:21.4487	valid-rmse:21.7298
[120]	train-rmse:21.0915	valid-rmse:21.3861
[140]	train-rmse:20.8463	valid-rmse:21.1721
[160]	train-rmse:20.6447	valid-rmse:20.9923
[180]	train-rmse:20.4683	valid-rmse:20.8174
[200]	train-rmse:20.3376	valid-rmse:20.6973
[220]	train-rmse:20.2268	valid-rmse:20.6042
[240]	train-rmse:20.1054	valid-rmse:20.4931
[249]	train-rmse:20.0539	valid-rmse:20.4438


Carregando semana 7 para fazer previsões

In [14]:
df7 = pd.read_csv('train_7.csv', dtype=dtype, usecols=usecols)

In [52]:
X = df7[['Cliente_ID', 'Producto_ID']]
y = df7['Demanda_uni_equil']

In [62]:
df7.head(10)

,Semana,Cliente_ID,Producto_ID,Demanda_uni_equil
0,7,1110,7,3301
1,7,1110,7,3301
2,7,1110,7,3301
3,7,1110,7,3301
4,7,1110,7,3301
5,7,1110,7,3301
6,7,1110,7,3301
7,7,1110,7,3301
8,7,1110,7,3301
9,7,1110,7,3301


In [53]:
X = MinMaxScaler().fit_transform(X)

In [54]:
X = xgb.DMatrix(X)

In [55]:
preds = model.predict(X)

In [56]:
preds

array([  26.0052,   26.0052,   26.0052, ..., 1033.8741, 1033.8741,
       1033.8741], dtype=float32)

In [57]:
from sklearn.metrics import mean_squared_log_error

np.sqrt(mean_squared_log_error(y, preds))

1.7673329